In [4]:
#pakagea
using DifferentialEquations;
using LaTeXStrings;
using Plots; pyplot(); # here, pyplot is used as a back-end to the Plots API
using Plots.PlotMeasures;
using BenchmarkTools;
using Combinatorics;
using ProgressMeter;
using DataFrames;
using GLM;
using LinearAlgebra;
using StatsBase;

ArgumentError: ArgumentError: Package PyPlot not found in current path:
- Run `import Pkg; Pkg.add("PyPlot")` to install the PyPlot package.


In [2]:
# Plot parameters
LW1 = 2.5
LW2 = 1.5
LW3 = 1
LS1 = :solid
LS2 = :dot
LS3 = :dash
LS4 = :dashdot
LA1 = 1
LA2 = 0.7
LA3 = 0.4
# File system paths
#
# Figures are stored in this path.
Figpath = "C:\\Users\\kolds\\Dropbox\\Master's_Thesis\\MSc_Report_Lyx\\fig\\Simulation_Results\\Jupyter_Notebook_3\\" 
;

In [5]:
#  Utility functions
# -- convert vector of samples to vector of timeseries
equal_lengths(v) = all(x->length(x)==length(first(v)), v)
#
function vec2vec(v)
    equal_lengths(v) || error("The element vectors must have equal lengths.")
    return [getindex.(v,i) for i in firstindex(v[1]):lastindex(v[1])]
end

;

In [6]:
# Parameters
const chpa = 1.15	# Specific heat capacity of air, kJ.kg-1.K-1
const chpw = 4.2	# Specific heat capacity of water, kJ.kg-1.K-1

const hAax = 55.6	# Heat transfer, air side heat exchanger, kW/K
const hAwx = 222	# Heat transfer, water side heat exchanger, kW/K
#
# Heat exchanger length
# const Lx = 1.0 # x = 0 to 1 , Simpler, gives a relative position and can be omitted from the heat exchanger model.
# Because it has a value of 1, it was omitted from the model.;


In [9]:
# Experimental ranges
n_exp = 3
n_Tw = n_exp
n_Ta = n_exp
n_mdw = n_exp
n_mda = n_exp
n_UAx = n_exp

r_Twc = range(4.,30.,length=n_Tw)
r_Tah = range(40.,100.,length=n_Ta)
r_mdw = range(20,200,length=n_mdw)
r_mda = range(20,200,length=n_mda)
r_UAx = range(20,200,length=n_UAx)
#
x_grid = [[Twc,Tah,mdw,mda,UAx] for Twc in r_Twc, Tah in r_Tah,mdw in r_mdw,mda in r_mda,UAx in r_UAx];

In [10]:
#solving models for all inputs
for i in 1:ngrid    
    Twc,Tah,mdw,mda,UAx = x_grid[i]
    u_hex = [Twc Tah mdw mda]   
    par_hex_a = [UAx chpw chpa]
    par_hex_n_Cp_T_dep = [UAx cp_w cp_a]
    sol_analytic = map(dx -> hex_a(u_hex, par_hex_a, dx), xspan_a)
    XAgrid[:,i] .= [sol_analytic[end][1],sol_analytic[1][end],mda,mdw,UAx]
    sol_numeric = hex_n_Cp_T_dep(u_hex, par_hex_n_Cp_T_dep, xspan_n)
    TNgrid[:,i] .= [sol_numeric[end][1],sol_numeric[1][end]]
end
;

UndefVarError: UndefVarError: ngrid not defined

In [11]:
X = XAgrid;
Y= TNgrid;

UndefVarError: UndefVarError: XAgrid not defined

In [12]:
#scaling
Xm = mean(X,dims=2);
Xstd = std(X,dims=2);
Xscale = [(X[i,:].-Xm[i])/Xstd[i] for i in 1:size(X,1)] |> x -> reduce(hcat,x) |> permutedims;
Ym = mean(Y,dims=2);
Ystd = std(Y,dims=2);
Yscale = [(Y[i,:].-Ym[i])/Ystd[i] for i in 1:size(Y,1)] |> x -> reduce(hcat,x) |> permutedims

UndefVarError: UndefVarError: mean not defined

In [13]:
#Estimating parameters
β= Yscale/Xscale

UndefVarError: UndefVarError: Yscale not defined

In [14]:
#calculating error for multiple order
nmax = 12
e1 = zeros(nmax)
for j in 1:nmax
    Phi = [phi_m(Xscale[:,i];n=j) for i in 1:size(Xscale,2)]  |> x -> reduce(hcat,x)
    println(cond(Phi))
    β= Yscale/Phi
    E1= Yscale-β*Phi
    e1[j] = norm(E1)/sqrt(length(Yscale))
end
   ; 

UndefVarError: UndefVarError: Xscale not defined

In [15]:
# buliding hybrid model
#combining analytic model with regression model
function timecalc(k)
    function hex_a(inputs,par,β) 
    Twc, Tah, mdw, mda,UAx = inputs
    chpw, chpa = par
    # Stanton numbers for air and water.
    NSta = UAx/chpa/mda	# Stanton number for air, -
    NStw = UAx/chpw/mdw	# Stanton number for water, -
    NStd = NStw - NSta	# Difference in Stanton numbers, -
    #
    Tac = (NStd*Tah + NSta*(1-exp(-NStd))*Twc)/(NStw-NSta*exp(-NStd))    
    Tw(x) = ((NStw*exp(-NStd*x)-NSta)*Twc+(NStw - NStw*exp(-NStd*x))*Tac)/NStd
    Ta(x) = ((NSta*exp(-NStd*x)-NSta)*Twc+(NStw - NSta*exp(-NStd*x))*Tac)/NStd
    Phi = [phi_m(Xscale[:,i];n=norder) for i in 1:size(Xscale,2)]  |> x -> reduce(hcat,x)
    Ys=β*Phi
    return [Ys[1] Ys[end]]
    end
end    

timecalc (generic function with 1 method)